In [85]:
# Definindo as bibliotecas 

import pandas as pd
import azure.storage.blob
import json
import os
import datetime
import io
from azure.storage.blob import PublicAccess, ContainerClient, BlobServiceClient, BlobClient

In [87]:
# Definindo as Funções

def create_obj(keys, value = ''):
    return { i: copy.copy(value) for i in keys }     

def download_blob(blob_client, blob_name):
    print('[{}]:[INFO] : Blob name: {}'.format(datetime.datetime.utcnow(), blob_name))
    print("[{}]:[INFO] : Downloading {} ...".format(datetime.datetime.utcnow(), blob_name))
    blob_data = blob_client.download_blob()
    print("[{}]:[INFO] : download finished".format(datetime.datetime.utcnow()))
    return blob_data
    
def upload_all_data(data, connection_string = connection_string, container_name = 'clean'):
    container_client = ContainerClient.from_connection_string(connection_string, container_name)
    print(container_client)
    print('Fazendo Upload arquivos...')

    for blob_name, blob_data in data.items():
        blob_client = container_client.get_blob_client(blob_name)
        blob_client.upload_blob(blob_data, overwrite = True)
        print('Upload finalizado: {}'.format(blob_name))

In [93]:
# JUNTA TODOS CSV'S

connection_string = "DefaultEndpointsProtocol=https;AccountName=staengdados;AccountKey=KtfGJ/u3NWxqsFBksx2gR8hRVAcpV0lsVr9liYwsXJoTx68DIa2KtFVobhO6Ob3bmo8PcobxzNYk+AStltMUjA==;EndpointSuffix=core.windows.net"
container_name = "clean"
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
container_client = blob_service_client.get_container_client(container_name)
blob_list = container_client.list_blobs()
tmp_data_frames = {}
for blob in blob_list:
    if blob.name.split('.')[1] != 'csv':
        continue
    if len(blob.name.split('_')) != 2:
        continue
    blob_client = container_client.get_blob_client(blob.name)
    data = download_blob(blob_client, blob.name)
    try:
        csv_string = data.content_as_text()
    except:
        csv_string = data.content_as_text(encoding='latin1')
    csvIO = io.StringIO(csv_string)
    csv_df = pd.read_csv(csvIO, sep=",")
    #csv_df.drop(columns = csv_df.columns[0], axis = 1, inplace = True)
    name_type_csv = blob.name.split('_')[-1]
    if name_type_csv in tmp_data_frames:
        tmp_data_frames.update({
            name_type_csv: pd.concat([tmp_data_frames[name_type_csv], csv_df])
        })
    else:
        tmp_data_frames.update({name_type_csv: csv_df})
tmp_data_frames = { k: v.to_csv(index=False) for k, v in tmp_data_frames.items() } 

[2022-08-15 20:49:18.734374]:[INFO] : Blob name: 1228436438229_ListarCategorias.csv
[2022-08-15 20:49:18.734374]:[INFO] : Downloading 1228436438229_ListarCategorias.csv ...
[2022-08-15 20:49:18.750004]:[INFO] : download finished
[2022-08-15 20:49:18.750004]:[INFO] : Blob name: 1228436438229_ListarClientesResumido.csv
[2022-08-15 20:49:18.750004]:[INFO] : Downloading 1228436438229_ListarClientesResumido.csv ...
[2022-08-15 20:49:18.765681]:[INFO] : download finished
[2022-08-15 20:49:18.781162]:[INFO] : Blob name: 1228436438229_ListarContasCorrentes.csv
[2022-08-15 20:49:18.781162]:[INFO] : Downloading 1228436438229_ListarContasCorrentes.csv ...
[2022-08-15 20:49:18.796785]:[INFO] : download finished
[2022-08-15 20:49:18.796785]:[INFO] : Blob name: 1228436438229_ListarDepatartamentos.csv
[2022-08-15 20:49:18.796785]:[INFO] : Downloading 1228436438229_ListarDepatartamentos.csv ...
[2022-08-15 20:49:18.812498]:[INFO] : download finished
[2022-08-15 20:49:18.812498]:[INFO] : Blob name: 122

[2022-08-15 20:49:19.611100]:[INFO] : download finished
[2022-08-15 20:49:19.617105]:[INFO] : Blob name: 1577300089365_ListarMovimentos.csv
[2022-08-15 20:49:19.618081]:[INFO] : Downloading 1577300089365_ListarMovimentos.csv ...
[2022-08-15 20:49:19.634968]:[INFO] : download finished


In [94]:
# FAZ O UPLOAD DOS CSV'S JUNTOS
connection_string = "DefaultEndpointsProtocol=https;AccountName=staengdados;AccountKey=KtfGJ/u3NWxqsFBksx2gR8hRVAcpV0lsVr9liYwsXJoTx68DIa2KtFVobhO6Ob3bmo8PcobxzNYk+AStltMUjA==;EndpointSuffix=core.windows.net"
container_name = "curated"

upload_all_data(tmp_data_frames, connection_string, container_name)

Fazendo Upload arquivos...
Upload finalizado: ListarCategorias.csv
Upload finalizado: ListarClientesResumido.csv
Upload finalizado: ListarContasCorrentes.csv
Upload finalizado: ListarDepatartamentos.csv
Upload finalizado: ListarMovimentos.csv
Upload finalizado: ListarLancCC.csv
